In [1]:
from diffeqpy import de
import numpy as np

In [3]:
# Arguments
is_restart = True
n_epoch = 10000
n_plot = 50
datasize = 50
tstep = 1
n_exp_train = 20
n_exp_test = 10
n_exp = n_exp_train + n_exp_test
noise = 0.05
ns = 6
nr = 3

#alg = AutoTsit5(Rosenbrock23(autodiff=false))
atol = 1e-6
rtol = 1e-3
lb = 1e-6
ub = 1e1

In [33]:
def trueODEfunc(dydt, y, k, t):
    # TG(1),ROH(2),DG(3),MG(4),GL(5),R'CO2R(6)
    r1 = k[0] * y[0] * y[1]
    r2 = k[1] * y[2] * y[1]
    r3 = k[2] * y[3] * y[1]
    dydt[0] = - r1  # TG
    dydt[1] = - r1 - r2 - r3  # TG
    dydt[2] = r1 - r2  # DG
    dydt[3] = r2 - r3  # MG
    dydt[4] = r3  # GL
    dydt[5] = r1 + r2 + r3  # R'CO2R
    #dydt[6] = 0

logA = np.array([18.60, 19.13, 7.93])
Ea = np.array([14.54, 14.42, 6.47])  # kcal/mol

def Arrhenius(logA, Ea, T):
    R = 1.98720425864083e-3
    k = np.exp(logA) * np.exp(-Ea / (R * T))
    return k



In [39]:
np.random.seed(1234)
u0_list = np.random.rand(n_exp, ns+1)
u0_list[:, 0:2] = 2.0 * u0_list[:, 0:2] + 0.2
u0_list[:, 2:ns] = 0
u0_list[:, ns] = u0_list[:, ns] * 20.0 + 323.0 # T[K]
tspan = [0.0, datasize * tstep]
tsteps = np.arange(tspan[0], tspan[1], (tspan[1]-tspan[0])/datasize)
ode_data_list = np.zeros((n_exp, ns, datasize))
yscale_list = []

def max_min(ode_data):
    return max(ode_data, dims=2) - min(ode_data, dims=2) + lb

#alg = de.AutoTsit5()

for i in range(0, n_exp):
    u0 = u0_list[i, :]
    k = Arrhenius(logA, Ea, u0[-1])
    prob_trueode = de.ODEProblem(trueODEfunc, u0, tspan, k)
    ode_data = de.solve(prob_trueode, saveat=tsteps)
    #ode_data_list[i, :, :] = ode_data

print(ode_data)


<PyCall.jlwrap OrdinaryDiffEq.ODECompositeSolution{Float64, 2, Vector{Vector{Float64}}, Nothing, Nothing, Vector{Float64}, Vector{Vector{Vector{Float64}}}, SciMLBase.ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, Vector{Float64}, SciMLBase.ODEFunction{true, PyObject, LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing}, Base.Iterators.Pairs{Union{}, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, SciMLBase.StandardODEProblem}, OrdinaryDiffEq.CompositeAlgorithm{Tuple{OrdinaryDiffEq.Tsit5{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!)}, OrdinaryDiffEq.Rosenbrock23{6, false, DiffEqBase.DefaultLinSolve, Val{:forward}}}, OrdinaryDiffEq.AutoSwitchCache{OrdinaryDiffEq.Tsit5{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!)}, OrdinaryDiffEq.Rosenbrock23{0, false, DiffEqBase.DefaultLinSolv